# **OVERVIEW OF THE DATASET**

- Definition of sleep event: 
    - Event class: Onset/Wakeup
    - Sleep Onset is indicate the beginning of the sleep, and wakeup is defined as the ebd of the sleep.
    - Single sleep period must be the longest inactive period while the accelerometer is worn. 
    - Single sleep must be the at leat 30 minutes in length which can be interrupted by bouts that do not exceed 30 consecutive 30 mm,inutes.
    
    
- The individual can have multiple sleep window in same clander day. But the **longest sleep window during the night will be recored**.


- There is no strict rule to limit number of sleep events in a givien period **however only one window assign per nigth**.


- The event that annotated when the accelerometer removed should be refrain from makking predictions. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

In [2]:
data = pd.read_parquet('/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet')
data.head(20)

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
5,038441c925bb,5,2018-08-14T15:30:25-0400,2.636700,0.0217
6,038441c925bb,6,2018-08-14T15:30:30-0400,2.636700,0.0217
7,038441c925bb,7,2018-08-14T15:30:35-0400,2.636700,0.0218
8,038441c925bb,8,2018-08-14T15:30:40-0400,2.798000,0.0223
9,038441c925bb,9,2018-08-14T15:30:45-0400,3.084700,0.0217


In [3]:
print(data.shape)
#print(data.night.nunique())

(127946340, 5)


In [4]:
data = pd.read_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet")

In [5]:
data.head(20)

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
5,038441c925bb,5,2018-08-14T15:30:25-0400,2.636700,0.0217
6,038441c925bb,6,2018-08-14T15:30:30-0400,2.636700,0.0217
7,038441c925bb,7,2018-08-14T15:30:35-0400,2.636700,0.0218
8,038441c925bb,8,2018-08-14T15:30:40-0400,2.798000,0.0223
9,038441c925bb,9,2018-08-14T15:30:45-0400,3.084700,0.0217


In [6]:
data.shape

(450, 5)